<a href="https://colab.research.google.com/github/carbonteamhb/eligibility/blob/main/clean_shapefiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon

In [ ]:
def remove_z_component(geometry):
    # Check if geometry is of type Polygon
    if isinstance(geometry, Polygon):
        # Remove Z component by stripping the third coordinate (z)
        coords = [(x, y) for x, y, z in geometry.exterior.coords]
        new_geom = Polygon(coords)
        return new_geom
    # Check if geometry is of type MultiPolygon
    elif isinstance(geometry, MultiPolygon):
        # Handle MultiPolygon geometries by recursively applying remove_z_component to each Polygon
        new_geoms = [remove_z_component(poly) for poly in geometry.geoms]
        return MultiPolygon(new_geoms)
    # Return the geometry as is if it doesn't match Polygon or MultiPolygon
    return geometry


# code to clean a shp that has z coordinates

File_shp = 'project_area_colombia_cc.shp'
gdf = gpd.read_file(File_shp).to_crs('EPSG:4326')

print(gdf.geometry.geom_type.unique())  # Check unique geometry types


# Extract individual polygons if needed (handling both Polygon and MultiPolygon)
polygons = gdf.geometry.apply(lambda x: list(x.geoms) if x.geom_type == 'MultiPolygon' else [x])

# Flatten the list of geometries
flattened_polygons = [poly for sublist in polygons for poly in sublist]

# Remove the Z component from each geometry by manually stripping the third coordinate
def remove_z_component(geometry):
    if isinstance(geometry, Polygon):
        # Remove Z by stripping the third coordinate from each point
        coords = [(x, y) for x, y, z in geometry.exterior.coords]
        new_geom = Polygon(coords)
        return new_geom
    elif isinstance(geometry, MultiPolygon):
        # Handle MultiPolygon geometries similarly
        new_geoms = [remove_z_component(poly) for poly in geometry.geoms]
        return MultiPolygon(new_geoms)
    return geometry


# Apply the function to remove Z component
clean_polygons = [remove_z_component(poly) for poly in flattened_polygons]

# Create a new GeoDataFrame with the cleaned geometries (2D only)
gdf_cleaned = gpd.GeoDataFrame(geometry=clean_polygons, crs=gdf.crs)


# Convert to Earth Engine
#File = geemap.geopandas_to_ee(gdf_cleaned)

# Save the cleaned GeoDataFrame to a shapefile
gdf_cleaned.to_file(f"{File_shp.replace('.shp', '')}_cleaned.shp")